In [1]:
import jsonlines

In [2]:
%%time
"""
What does the file look like?
"""
with open("../data/train.jsonl", "r") as file:
    for i, line in enumerate(file):
        if i > 0:
            break
        print(line[:1000])

{"session":0,"events":[{"aid":1517085,"ts":1659304800025,"type":"clicks"},{"aid":1563459,"ts":1659304904511,"type":"clicks"},{"aid":1309446,"ts":1659367439426,"type":"clicks"},{"aid":16246,"ts":1659367719997,"type":"clicks"},{"aid":1781822,"ts":1659367871344,"type":"clicks"},{"aid":1152674,"ts":1659367885796,"type":"clicks"},{"aid":1649869,"ts":1659369893840,"type":"carts"},{"aid":461689,"ts":1659369898050,"type":"carts"},{"aid":305831,"ts":1659370027105,"type":"orders"},{"aid":461689,"ts":1659370027105,"type":"orders"},{"aid":362233,"ts":1659370064916,"type":"clicks"},{"aid":1649869,"ts":1659370067686,"type":"clicks"},{"aid":1649869,"ts":1659371003682,"type":"clicks"},{"aid":984597,"ts":1659371033243,"type":"clicks"},{"aid":1649869,"ts":1659371042297,"type":"clicks"},{"aid":803544,"ts":1659371044075,"type":"clicks"},{"aid":1110941,"ts":1659371104329,"type":"clicks"},{"aid":1190046,"ts":1659371123063,"type":"clicks"},{"aid":1760685,"ts":1659375531385,"type":"clicks"},{"aid":631008,"ts"

In [ ]:
%%time
"""
How long does it take to scan and parse the whole file in Python?
"""
# Open file in read-only mode
with jsonlines.open("../data/train.jsonl", "r") as reader:
    # Iterate over parsed JSON objects
    for i, obj in enumerate(reader):
        if i % 1_000_000==0:
            print(f"Object: {i:,d}")